<h1><center>Исследование данных сервиса “Яндекс.Музыка” — сравнение пользователей двух городов</center></h1>

# Описание задачи
На реальных данных музыкального сервиса c помощью библиотеки Pandas и её возможностей требуется проверить данные и сравнить поведение и предпочтения пользователей двух столиц — Москвы и Санкт-Петербурга.
  
**Цель исследования** — проверить три гипотезы:
* Активность пользователей зависит от дня недели и различается в Москве и Петербурге. 
* В зависимости от дня недели (понедельник, среда или пятница) и времени суток в Москве преобладают одни жанры, а в Петербурге – другие.
* В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

**Ход исследования**

1. [Обзор данных.](#Обзор-данных)
2. [Предобработка данных.](#Предобработка-данных)
3. [Проверка гипотез.](#Проверка-гипотез)

## Обзор данных

In [1]:
# Импорт библиотеки pandas

import pandas as pd

In [2]:
# чтение файла

df = pd.read_csv('/datasets/yandex_music_project.csv')

In [3]:
# знакомство с содержанием

print(df.head(10))

     userID                        Track            artist   genre  \
0  FFB692EC            Kamigata To Boots  The Mass Missile    rock   
1  55204538  Delayed Because of Accident  Andreas Rönnberg    rock   
2    20EC38            Funiculì funiculà       Mario Lanza     pop   
3  A3DD03C9        Dragons in the Sunset        Fire + Ice    folk   
4  E2DC1FAE                  Soul People        Space Echo   dance   
5  842029A1                    Преданная         IMPERVTOR  rusrap   
6  4CB90AA5                         True      Roman Messer   dance   
7  F03E1C1F             Feeling This Way   Polina Griffith   dance   
8  8FA1D3BE     И вновь продолжается бой               NaN  ruspop   
9  E772D5C0                    Pessimist               NaN   dance   

             City        time        Day  
0  Saint-Petersburg  20:28:33  Wednesday  
1            Moscow  14:07:09     Friday  
2  Saint-Petersburg  20:58:07  Wednesday  
3  Saint-Petersburg  08:37:09     Monday  
4            M

In [4]:
# получение общей информации

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


Из документации известно:

* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.  
Также в названиях колонок видны нарушения стиля:
* Строчные буквы сочетаются с прописными.
* Встречаются пробелы.
* Не используется "змеиный регистр", вместо чего используется "верблюжий" в колонке userID.

**Выводы**

В каждой строке таблицы — данные о прослушанном треке. Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку. 

Предварительно можно утверждать, что данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а в названиях колонок — расхождения с хорошим стилем.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Предобработка данных

### Стиль заголовков

In [6]:
# вывод наименований столбцов

print(df.columns)

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')


In [7]:
# переименуем столбцы для удобства

df = df.rename(columns = {'  userID': 'user_id', 'Track': 'track', '  City  ': 'city', 'Day': 'day'})

In [8]:
# проверка

print(df.columns)

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')


### Пропуски значений

In [9]:
# вывод значений пропусков в каждом столбце

print(df.isna().sum())

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64


Не все пропущенные значения влияют на исследование. Так в `track` и `artist` пропуски не важны для вашей работы. Достаточно заменить их явными обозначениями.

Но пропуски в `genre` могут помешать сравнению музыкальных вкусов в Москве и Санкт-Петербурге. На практике было бы правильно установить причину пропусков и восстановить данные. Такой возможности нет в учебном проекте. Придётся:
* заполнить и эти пропуски явными обозначениями;
* оценить, насколько они повредят расчётам.

In [10]:
# замена пропущенных значений на категорию unknown

columns_to_replace = ['track', 'artist', 'genre']

for col in columns_to_replace:
    df[:][col] = df[:][col].fillna('unknown')

In [11]:
# проверка

print(df.isna().sum())

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64


### Дубликаты

In [12]:
# определение кол-ва явных дубликатов

df.duplicated().sum()

3826

In [13]:
# удаление явных дубликатов

df = df.drop_duplicates().reset_index(drop=True)

In [14]:
# проверка

df.duplicated().sum()

0

Следующий шаг – избавление от неявных дубликатов. Например, название одного и того же жанра может быть записано немного по-разному. Это не допустимо и может исказить результат исследования.

In [15]:
# вывод уникальных значений из столбца genre

genre_sorted = df['genre'].sort_values()
genre_sorted.unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

Мы видим несколько разных значений которые отсылают к жанру hiphop

In [16]:
# замена неявных дубликатов жанра hip-hop

df['genre'] = df['genre'].replace(['hip', 'hop', 'hip-hop'], 'hiphop')

In [17]:
# проверка

genre_sorted = df['genre'].sort_values()
print(genre_sorted.unique())

['acid' 'acoustic' 'action' 'adult' 'africa' 'afrikaans' 'alternative'
 'alternativepunk' 'ambient' 'americana' 'animated' 'anime' 'arabesk'
 'arabic' 'arena' 'argentinetango' 'art' 'audiobook' 'author' 'avantgarde'
 'axé' 'baile' 'balkan' 'beats' 'bigroom' 'black' 'bluegrass' 'blues'
 'bollywood' 'bossa' 'brazilian' 'breakbeat' 'breaks' 'broadway'
 'cantautori' 'cantopop' 'canzone' 'caribbean' 'caucasian' 'celtic'
 'chamber' 'chanson' 'children' 'chill' 'chinese' 'choral' 'christian'
 'christmas' 'classical' 'classicmetal' 'club' 'colombian' 'comedy'
 'conjazz' 'contemporary' 'country' 'cuban' 'dance' 'dancehall' 'dancepop'
 'dark' 'death' 'deep' 'deutschrock' 'deutschspr' 'dirty' 'disco' 'dnb'
 'documentary' 'downbeat' 'downtempo' 'drum' 'dub' 'dubstep' 'eastern'
 'easy' 'electronic' 'electropop' 'emo' 'entehno' 'epicmetal' 'estrada'
 'ethnic' 'eurofolk' 'european' 'experimental' 'extrememetal' 'fado'
 'fairytail' 'film' 'fitness' 'flamenco' 'folk' 'folklore' 'folkmetal'
 'folkrock' 

**Выводы**

Предобработка обнаружила три проблемы в данных:

- нарушения в стиле заголовков,
- пропущенные значения,
- дубликаты — явные и неявные.

Были исправлены заголовки для удобства работы с таблицей. Исправлена проблема дубликатов для улучшения точности исследования.
Пропуски заменены на `'unknown'` для учёта в анализе. Ещё предстоит увидеть, не повредят ли исследованию пропуски в колонке `genre`.

Переходим к проверке гипотез.

## Проверка гипотез

### Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге.
Для этого следует:

* Разделить пользователей Москвы и Санкт-Петербурга.
* Сравнить, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.

In [18]:
# группируем активность по городу и подсчитываем кол-во записей

city_group = df.groupby('city')['day']
city_group.count()

city
Moscow              42741
Saint-Petersburg    18512
Name: day, dtype: int64

В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.

In [19]:
# группировка данных и подсчёт прослушиваний по дню

day_group = df.groupby('day')['track']
print(day_group.count())

day
Friday       21840
Monday       21354
Wednesday    18059
Name: track, dtype: int64


В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

Для этого примените последовательную фильтрацию с логической индексацией (или сложные логические выражения в одну строку, если вы уже знакомы с ними).

Затем посчитайте значения в столбце `user_id` получившейся таблицы. Результат сохраните в новую переменную. Верните эту переменную из функции.

In [39]:
def number_tracks(day, city):
'''
Функция возвращает кол-во прослушиваний в заданном городе в конкретный день

day - день недели
city - название города
'''
    track_list = df[(df['day'] == day) & (df['city'] == city)] 
    track_list_count = track_list['user_id'].count()
    print(track_list['user_id'])
    return track_list_count

Прогоним значения нужных дней и городов через функцию

In [40]:
number_tracks('Monday', 'Moscow') # количество прослушиваний в Москве по понедельникам 

4        E2DC1FAE
10       BC5A3A29
11       8B5192C0
12       FF3FD2BD
15       E3C5756F
           ...   
61213    3D9BF4F3
61219    57B3F0CE
61224    9656377C
61225    85A7CFD3
61247    83A474E7
Name: user_id, Length: 15740, dtype: object


15740

In [22]:
number_tracks('Monday', 'Saint-Petersburg') # количество прослушиваний в Санкт-Петербурге по понедельникам

5614


5614

In [23]:
number_tracks('Wednesday', 'Moscow') # количество прослушиваний в Москве по средам

11056


11056

In [24]:
number_tracks('Wednesday', 'Saint-Petersburg') # количество прослушиваний в Санкт-Петербурге по средам

7003


7003

In [25]:
number_tracks('Friday', 'Moscow') # количество прослушиваний в Москве по пятницам 

15945


15945

In [26]:
number_tracks('Friday', 'Saint-Petersburg') # количество прослушиваний в Санкт-Петербурге по пятницам

5895


5895

In [27]:
# свдоим полученные данные в таблицу

track_listening = pd.DataFrame(data=[['Moscow', 15740, 11056, 15945,],
                                    ['Saint-Petersburg', 5614, 7003, 5895]], columns=['city', 'monday', 'wednesday', 'friday'])
print(track_listening)

               city  monday  wednesday  friday
0            Moscow   15740      11056   15945
1  Saint-Petersburg    5614       7003    5895


**Выводы**

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы.

### Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

In [28]:
# таблица с данными по Москве

moscow_general = df[df['city'] == 'Moscow'] 

In [29]:
# таблица с данными по Петербургу

spb_general = df[df['city'] == 'Saint-Petersburg']

Создадим функцию `genre_weekday()`

In [30]:
def genre_weekday(table, day, time1, time2):
    '''
    Функция возвращает информацию о топ-10 жанров треков, которые прослушивали в указанный день,
    в указанном временном промежутке
    
    table - датафрейм
    day - день недели
    time1 - начальная временная метка
    time2 - конечная временная метка
    '''
    
    genre_df = table.loc[table.loc[:, 'day'] == day] 
    genre_df = genre_df.loc[genre_df.loc[:, 'time'] < time2] 
    genre_df = genre_df.loc[genre_df.loc[:, 'time'] > time1] 
    genre_df_grouped = genre_df.groupby('genre')['genre'].count() 
    genre_df_sorted = genre_df_grouped.sort_values(ascending=False) 
    
    return genre_df_sorted[:10]

Cравним результаты функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [31]:
# Москва, утро понедельника

print(genre_weekday(moscow_general, 'Monday', '07:00', '11:00'))

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64


In [32]:
# Петербург, утро понедельника

print(genre_weekday(spb_general, 'Monday', '07:00', '11:00'))

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64


In [33]:
# Москва, вечер пятницы

print(genre_weekday(moscow_general, 'Friday', '17:00', '23:00'))

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64


In [34]:
# Петербург, вечер пятницы

print(genre_weekday(spb_general, 'Friday', '17:00', '23:00'))

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64


**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

In [35]:
# датафрейм с сортированным списком прослушивания жанров в Москве 

moscow_genres = moscow_general.groupby('genre')['genre'].count().sort_values(ascending=False)

In [36]:
# просмотр первых 10 строк moscow_genres

print(moscow_genres.head(10)) 

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64


In [37]:
# датафрейм с сортированным списком прослушивания жанров в Петербурге

spb_genres = spb_general.groupby('genre')['genre'].count().sort_values(ascending=False)

In [38]:
# просмотр первых 10 строк spb_genres

print(spb_genres.head(10))

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64


**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге. 


## Итоги исследования

Выполнена проверка трёх гипотез и получены следующие результаты:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге. 

Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.

**На практике исследования содержат проверки статистических гипотез.**
Из данных одного сервиса не всегда можно сделать вывод о всех жителях города.
Проверки статистических гипотез покажут, насколько они достоверны, исходя из имеющихся данных.